<a href="https://colab.research.google.com/github/bvarel/Liar-NN/blob/main/Machine_Learning_Final_Project_DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Only Run these if needed
!pip install datasets

!pip install transformers

from datasets import load_dataset
import pandas as pd
import nltk
import re
import string
import os
import random
import itertools
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

from transformers import TFAutoModel, AutoTokenizer, BertConfig, TFBertModel
from transformers import DistilBertModel, DistilBertConfig,BertTokenizerFast

import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
#pd.set_option('display.max_colwidth', None)
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
BATCH_SIZE = 16
N_EPOCHS = 3


dataset = load_dataset("liar")

     |████████████████████████████████| 312 kB 3.9 MB/s 
     |████████████████████████████████| 134 kB 55.7 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 212 kB 40.3 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 127 kB 39.6 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 53.7 MB/s 
     |████████████████████████████████| 144 kB 38.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 3.8 MB 4.0 MB/s 


Downloading:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset liar downloaded and prepared to /root/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])
validation= pd.DataFrame(dataset['validation'])


train_df = train.fillna('None')
test_df = test.fillna('None')
val_df = validation.fillna('None')

train = train_df.values
test = test_df.values
val = val_df.values

In [3]:
#Labeling 1 or 0 for true or not true
def label_true (row):
    if row['label'] < 3 :
        return 0
    return 1

train_df['true_false'] = train_df.apply (lambda row: label_true(row), axis=1)
test_df['true_false'] = test_df.apply (lambda row: label_true(row), axis=1)
val_df['true_false'] = val_df.apply (lambda row: label_true(row), axis=1)

In [4]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def convert_to_transformer_inputs(str1, str2, tokenizer, max_sequence_length, double=True):
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        #input_segments = inputs["token_type_ids"]
        
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        #input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks]
    
    if double:
    
        input_ids_1, input_masks_1 = return_id(
            str1, None, 'longest_first', max_sequence_length)

        input_ids_2, input_masks_2, input_segments_2 = return_id(
            str2, None, 'longest_first', max_sequence_length)

        return [input_ids_1, input_masks_1, 
                input_ids_2, input_masks_2]
    
    else:
        
        input_ids, input_masks = return_id(
            str1, str2, 'longest_first', max_sequence_length)

        return [input_ids, input_masks,
                None, None]        



In [5]:
def compute_input_arrays(df, columns, tokenizer, max_sequence_length, double=True):
    
    input_ids_1, input_masks_1 = [], []
    input_ids_2, input_masks_2= [], []
    for _, instance in tqdm(df[columns].iterrows(), total=len(df)):
        str1, str2 = instance[columns[0]], instance[columns[1]]

        ids_1, masks_1, ids_2, masks_2 = \
        convert_to_transformer_inputs(str1, str2, tokenizer, max_sequence_length, double=double)
        
        input_ids_1.append(ids_1)
        input_masks_1.append(masks_1)

        input_ids_2.append(ids_2)
        input_masks_2.append(masks_2)
        
    if double:
        
        return [np.asarray(input_ids_1, dtype=np.int32), 
                np.asarray(input_masks_1, dtype=np.int32), 
                np.asarray(input_ids_2, dtype=np.int32), 
                np.asarray(input_masks_2, dtype=np.int32)] 

    
    else:
        
        return [np.asarray(input_ids_1, dtype=np.int32), 
                np.asarray(input_masks_1, dtype=np.int32)]


In [6]:
labels = {'train':[train[i][1] for i in range(len(train))], 'test':[test[i][1] for i in range(len(test))], 'val':[val[i][1] for i in range(len(val))]}
statements = {'train':[train[i][2] for i in range(len(train))], 'test':[test[i][2] for i in range(len(test))], 'val':[val[i][2] for i in range(len(val))]}
subjects = {'train':[train[i][3] for i in range(len(train))], 'test':[test[i][3] for i in range(len(test))], 'val':[val[i][3] for i in range(len(val))]}
speakers = {'train':[train[i][4] for i in range(len(train))], 'test':[test[i][4] for i in range(len(test))], 'val':[val[i][4] for i in range(len(val))]}
jobs = {'train':[train[i][5] for i in range(len(train))], 'test':[test[i][5] for i in range(len(test))], 'val':[val[i][5] for i in range(len(val))]}
states = {'train':[train[i][6] for i in range(len(train))], 'test':[test[i][6] for i in range(len(test))], 'val':[val[i][6] for i in range(len(val))]}
affiliations = {'train':[train[i][7] for i in range(len(train))], 'test':[test[i][7] for i in range(len(test))], 'val':[val[i][7] for i in range(len(val))]}
credits = {'train':[train[i][8:13] for i in range(len(train))], 'test':[test[i][8:13] for i in range(len(test))], 'val':[val[i][8:13] for i in range(len(val))]}
contexts= {'train':[train[i][13] for i in range(len(train))], 'test':[test[i][13] for i in range(len(test))], 'val':[val[i][13] for i in range(len(val))]}
#contexts = {'train':[train[i][14] for i in range(len(train))], 'test':[test[i][14] for i in range(len(test))], 'val':[val[i][14] for i in range(len(val))]}
#justification = {'train':[train[i][15] for i in range(len(train))], 'test':[test[i][15] for i in range(len(test))], 'val':[val[i][15] for i in range(len(val))]}

In [7]:
metadata = {'train':[0]*len(train), 'val':[0]*len(val), 'test':[0]*len(test)}


for i in range(len(train)):
    subject = subjects['train'][i]
    if subject == 0:
        subject = 'None'
    
    speaker = speakers['train'][i]
    if speaker == 0:
        speaker = 'None'
    
    job = jobs['train'][i]
    if job == 0:
        job = 'None'
    
    state = states['train'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['train'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['train'][i]
    if context== 0 :
        context = 'None'
        
    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context
    
    #subject_train.append(subject)
    #speaker_train.append(speaker)
    #job_train.append(job)
    #state_train.append(state)
    #affiliation_train.append(affiliation)
    #context_train.append(context)
    
    metadata['train'][i] = meta
    
for i in range(len(val)):
    subject = subjects['val'][i]
    if subject == 0:
        subject = 'None'
    
    speaker = speakers['val'][i]
    if speaker == 0:
        speaker = 'None'
    
    job = jobs['val'][i]
    if job == 0:
        job = 'None'
    
    state = states['val'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['val'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['val'][i]
    if context == 0 :
        context = 'None'
        
    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context
    
    metadata['val'][i] = meta
    
for i in range(len(test)):
    subject = subjects['test'][i]
    if subject == 0:
        subject = 'None'
    
    speaker = speakers['test'][i]
    if speaker == 0:
        speaker = 'None'
    
    job = jobs['test'][i]
    if job == 0:
        job = 'None'
    
    state = states['test'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['test'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['test'][i]
    if context == 0 :
        context = 'None'
        
    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context
    
    metadata['test'][i] = meta

In [8]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=25)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize = 14)

    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label', fontsize=20)

In [9]:
train_df['metadata']= metadata['train']
test_df['metadata']= metadata['test']
val_df['metadata']= metadata['val']

In [10]:
X_train_statement = train_df['statement']
X_test_statement = test_df['statement']
X_val_statement = val_df['statement']

y_train = train_df['true_false']
y_val = val_df['true_false']
y_test = test_df['true_false']

print(X_train_statement.shape, X_test_statement.shape, X_val_statement.shape)
print(y_train.shape, y_test.shape,y_val.shape)

(10269,) (1283,) (1284,)
(10269,) (1283,) (1284,)


In [11]:
MAX_SEQUENCE_LENGTH = 300
#maybe try 128?

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)#AutoTokenizer.from_pretrained("bert-base-uncased", padding=True, truncation=True)

input_train = compute_input_arrays(train_df, ['statement','metadata'], tokenizer, MAX_SEQUENCE_LENGTH, double=False)
input_val = compute_input_arrays(val_df, ['statement','metadata'], tokenizer, MAX_SEQUENCE_LENGTH, double=False)
input_test = compute_input_arrays(test_df, ['statement','metadata'], tokenizer, MAX_SEQUENCE_LENGTH, double=False)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

  0%|          | 0/10269 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/1284 [00:00<?, ?it/s]

  0%|          | 0/1283 [00:00<?, ?it/s]

In [12]:
map_label = dict(enumerate(train_df.true_false.factorize()[1]))
map_label

{0: 0, 1: 1}

In [13]:
def simple_distil_bert():
    
    set_seed(33)
    
    opt = Adam(learning_rate=2e-5)
    
    id_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
 
    distil_model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
    
    embedding = distil_model(id_, attention_mask=mask_)[0]
    

    dense = Dense(64,activation='relu')(embedding)
    dropout= Dropout(0.2)(dense)
    
    
    out = Dense(len(map_label), activation='softmax')(dropout)

    model = Model(inputs=[id_, mask_], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [14]:
model_1 = simple_distil_bert()
model_1.fit(input_train, y_train,validation_data = [input_val,y_val], epochs=3, batch_size = 32)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Epoch 1/3
321/321 [==============================] - 213s 629ms/step - loss: 0.6974 - accuracy: 0.5053 - val_loss: 0.6906 - val_accuracy: 0.5935
Epoch 2/3
321/321 [==============================] - 200s 623ms/step - loss: 0.6896 - accuracy: 0.5929 - val_loss: 0.6873 - val_accuracy: 0.5981
Epoch 3/3
321/321 [==============================] - 200s 623ms/step - loss: 0.6863 - accuracy: 0.5982 - val_loss: 0.6831 - val_accuracy: 0.6075


In [15]:
### PREDICT TEST ###

pred_test = np.argmax(model_1.predict(input_test), axis=1)

print(classification_report([map_label[i] for i in y_test], [map_label[i] for i in pred_test]))

              precision    recall  f1-score   support

           0       0.61      0.98      0.75       766
           1       0.67      0.05      0.10       517

    accuracy                           0.61      1283
   macro avg       0.64      0.52      0.42      1283
weighted avg       0.63      0.61      0.49      1283



In [ ]:
#Confusion Matrix
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=25)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize = 14)

    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label', fontsize=20)


In [ ]:
#Plot Confusion Matrix
cnf_matrix = confusion_matrix([map_label[i] for i in y_test], 
                              [map_label[i] for i in pred_test])

plt.figure(figsize=(2,2))
plot_confusion_matrix(cnf_matrix, classes=list(map_label.values()))
plt.show()

In [16]:
def simple_distil_bert_2():
    
    set_seed(33)
    
    opt = Adam(learning_rate=2e-5)
    
    id_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)

    distil_model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
    
    embedding = distil_model(id_, attention_mask=mask_)[0]
    

    dense = Dense(64,activation='relu')(embedding)

    dropout= Dropout(0.5)(dense)


    
    out = Dense(len(map_label), activation='softmax')(dropout)

    model = Model(inputs=[id_, mask_], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [17]:
model_2 = simple_distil_bert_2()
model_2.fit(input_train, y_train,validation_data = [input_val,y_val], epochs=3, batch_size = 32)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_40']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
321/321 [==============================] - 212s 629ms/step - loss: 0.6990 - accuracy: 0.4963 - val_loss: 0.6914 - val_accuracy: 0.5958
Epoch 2/3
321/321 [==============================] - 200s 623ms/step - loss: 0.6903 - accuracy: 0.5789 - val_loss: 0.6884 - val_accuracy: 0.5927
Epoch 3/3
321/321 [==============================] - 200s 624ms/step - loss: 0.6875 - accuracy: 0.5921 - val_loss: 0.6854 - val_accuracy: 0.5935


In [18]:
### PREDICT TEST ###

pred_test = np.argmax(model_2.predict(input_test), axis=1)

print(classification_report([map_label[i] for i in y_test], [map_label[i] for i in pred_test]))

              precision    recall  f1-score   support

           0       0.60      1.00      0.75       766
           1       0.50      0.00      0.00       517

    accuracy                           0.60      1283
   macro avg       0.55      0.50      0.38      1283
weighted avg       0.56      0.60      0.45      1283



In [19]:
def simple_distil_bert_3():
    
    set_seed(33)
    
    opt = Adam(learning_rate=2e-5)
    
    id_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)

    distil_model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
    
    embedding = distil_model(id_, attention_mask=mask_)[0]

    dense = Dense(512,activation='relu')(embedding)
    
    dropout= Dropout(0.2)(dense)
 
    out = Dense(len(map_label), activation='softmax')(dropout)

    model = Model(inputs=[id_, mask_], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [20]:
model_3 = simple_distil_bert_3()
model_3.fit(input_train, y_train,validation_data = [input_val,y_val], epochs=3, batch_size = 32)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_61']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
321/321 [==============================] - 212s 629ms/step - loss: 0.6791 - accuracy: 0.5903 - val_loss: 0.6720 - val_accuracy: 0.6106
Epoch 2/3
321/321 [==============================] - 200s 623ms/step - loss: 0.6632 - accuracy: 0.6086 - val_loss: 0.6825 - val_accuracy: 0.5709
Epoch 3/3
321/321 [==============================] - 200s 624ms/step - loss: 0.6198 - accuracy: 0.6592 - val_loss: 0.6966 - val_accuracy: 0.5693


In [21]:
### PREDICT TEST ###

pred_test = np.argmax(model_3.predict(input_test), axis=1)

print(classification_report([map_label[i] for i in y_test], [map_label[i] for i in pred_test]))

              precision    recall  f1-score   support

           0       0.62      0.82      0.71       766
           1       0.49      0.26      0.34       517

    accuracy                           0.59      1283
   macro avg       0.56      0.54      0.52      1283
weighted avg       0.57      0.59      0.56      1283



In [22]:
def simple_distil_bert_4():
    
    set_seed(33)
    
    opt = Adam(learning_rate=2e-5)
    
    id_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)

    distil_model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
    
    embedding = distil_model(id_, attention_mask=mask_)[0]
    

    dense = Dense(512,activation='relu')(embedding)
    

    dropout= Dropout(0.2)(dense)

    dense_2 = Dense(128,activation='relu')(dropout)

    dropout_2 = Dropout(0.2)(dense_2)

    
    out = Dense(len(map_label), activation='softmax')(dropout_2)

    model = Model(inputs=[id_, mask_], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [23]:
model_4 = simple_distil_bert_4()
model_4.fit(input_train, y_train,validation_data = [input_val,y_val], epochs=3, batch_size = 32)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_82']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
321/321 [==============================] - 212s 630ms/step - loss: 0.6798 - accuracy: 0.5854 - val_loss: 0.6718 - val_accuracy: 0.5935
Epoch 2/3
321/321 [==============================] - 200s 623ms/step - loss: 0.6660 - accuracy: 0.6010 - val_loss: 0.6739 - val_accuracy: 0.5989
Epoch 3/3
321/321 [==============================] - 200s 623ms/step - loss: 0.6086 - accuracy: 0.6693 - val_loss: 0.7526 - val_accuracy: 0.5974


In [24]:
### PREDICT TEST ###

pred_test = np.argmax(model_4.predict(input_test), axis=1)

print(classification_report([map_label[i] for i in y_test], [map_label[i] for i in pred_test]))

              precision    recall  f1-score   support

           0       0.62      0.86      0.72       766
           1       0.51      0.22      0.31       517

    accuracy                           0.60      1283
   macro avg       0.56      0.54      0.51      1283
weighted avg       0.57      0.60      0.55      1283



In [25]:
 model_4.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 tf_distil_bert_for_sequence_cl  TFSequenceClassifie  66955010   ['input_7[0][0]',                
 assification_3 (TFDistilBertFo  rOutput(loss=None,               'input_8[0][0]']                
 rSequenceClassification)       logits=(None, 2),                                                 
                                 hidden_states=None                                         

In [26]:
model_3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 tf_distil_bert_for_sequence_cl  TFSequenceClassifie  66955010   ['input_5[0][0]',                
 assification_2 (TFDistilBertFo  rOutput(loss=None,               'input_6[0][0]']                
 rSequenceClassification)       logits=(None, 2),                                                 
                                 hidden_states=None                                         

In [27]:
model_2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 tf_distil_bert_for_sequence_cl  TFSequenceClassifie  66955010   ['input_3[0][0]',                
 assification_1 (TFDistilBertFo  rOutput(loss=None,               'input_4[0][0]']                
 rSequenceClassification)       logits=(None, 2),                                                 
                                 hidden_states=None                                         

In [28]:
model_1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 tf_distil_bert_for_sequence_cl  TFSequenceClassifie  66955010   ['input_1[0][0]',                
 assification (TFDistilBertForS  rOutput(loss=None,               'input_2[0][0]']                
 equenceClassification)         logits=(None, 2),                                                 
                                 hidden_states=None                                           